# Mischievous pricing conventions

(Based on [a question](http://stackoverflow.com/questions/15273797/) by _Stack Exchange_ user ducky. Thanks!)

In [1]:
from QuantLib import *
import pandas as pd

#### The case of the bond off par

Like our user, I'll instantiate a four-years floating-rate bond with three-months coupons. It's being issued on the evaluation date, January 5th 2010, and for simplicity I won't use any settlement days or holidays:

In [2]:
today = Date(5, January, 2010)
Settings.instance().evaluationDate = today

discounting_curve = RelinkableYieldTermStructureHandle()
forecasting_curve = RelinkableYieldTermStructureHandle()

index = USDLibor(Period(3, Months), forecasting_curve)

settlement_days = 0
calendar = NullCalendar()

face_amount = 100.0
schedule = Schedule(today, today + Period(4, Years),
                    Period(3, Months), calendar,
                    Unadjusted, Unadjusted,
                    DateGeneration.Forward, False)

bond = FloatingRateBond(settlement_days,
                        face_amount,
                        schedule,
                        index,
                        Thirty360(),
                        Unadjusted,
                        fixingDays = 0)

bond.setPricingEngine(DiscountingBondEngine(discounting_curve))

To price it, we use a flat 10% quarterly rate for both forecasting and discounting...

In [3]:
flat_rate = FlatForward(today, 0.10, Thirty360(), Compounded, Quarterly)
forecasting_curve.linkTo(flat_rate)
discounting_curve.linkTo(flat_rate)

...so we expect the bond to be at par. Is it?

In [4]:
print(bond.cleanPrice())

99.5433545426823


Hmm.

#### What is happening here?

We have mismatched a few conventions. The ones with the largest effect are the day-count conventions used for the curve and the index. Here they are:

In [5]:
print(flat_rate.dayCounter())
print(as_coupon(bond.cashflows()[0]).dayCounter())
print(index.dayCounter())

30/360 (Bond Basis) day counter
30/360 (Bond Basis) day counter
Actual/360 day counter


Thus, the coupons accrue for the expected time (given by their day-count convention); however, the rates are not the expected 10%. They are calculated from discount factors given by the curve according to its 30/360 convention and recombined by the index according to its Actual/360 convention, which doesn't end well.

In [6]:
coupons = [ as_coupon(c) for c in bond.cashflows()[:-1] ]
pd.DataFrame([(c.date(), c.rate(), c.accrualPeriod())
              for c in coupons ],
             columns=('Date', 'Rate', 'Accrual period'),
             index=range(1,len(coupons)+1))

,Date,Rate,Accrual period
1,"April 5th, 2010",0.100014,0.25
2,"July 5th, 2010",0.098901,0.25
3,"October 5th, 2010",0.097826,0.25
4,"January 5th, 2011",0.097826,0.25
5,"April 5th, 2011",0.100000,0.25
6,"July 5th, 2011",0.098901,0.25
7,"October 5th, 2011",0.097826,0.25
8,"January 5th, 2012",0.097899,0.25
9,"April 5th, 2012",0.098952,0.25
10,"July 5th, 2012",0.098849,0.25


#### The importance of being consistent

In order to reproduce the textbook value, we have to reconcile the different conventions (which are, well, conveniently glossed over in textbooks). The correct one to choose depends on the terms and conditions of the bond; it is likely to be the Actual/360 convention used by the USD libor, so we'll pass it to both the bond and the curve:

In [7]:
bond = FloatingRateBond(settlement_days,
                        face_amount,
                        schedule,
                        index,
                        Actual360(),
                        Unadjusted,
                        fixingDays = 0)

bond.setPricingEngine(DiscountingBondEngine(discounting_curve))

In [8]:
flat_rate_2 = FlatForward(today, 0.10, Actual360(), Compounded, Quarterly)
forecasting_curve.linkTo(flat_rate_2)
discounting_curve.linkTo(flat_rate_2)

In [9]:
print(bond.cleanPrice())

100.00117521248728


There's still a small discrepancy, which is likely due to date adjustments in the underlying USD libor fixings. The coupon rates are much better overall, so we seem to be on the right track.

In [10]:
coupons = [ as_coupon(c) for c in bond.cashflows()[:-1] ]
pd.DataFrame([(c.date(), c.rate(), c.accrualPeriod())
              for c in coupons ],
             columns=('Date', 'Rate', 'Accrual period'),
             index=range(1,len(coupons)+1))

,Date,Rate,Accrual period
1,"April 5th, 2010",0.100014,0.250000
2,"July 5th, 2010",0.100014,0.252778
3,"October 5th, 2010",0.100028,0.255556
4,"January 5th, 2011",0.100028,0.255556
5,"April 5th, 2011",0.100000,0.250000
6,"July 5th, 2011",0.100014,0.252778
7,"October 5th, 2011",0.100028,0.255556
8,"January 5th, 2012",0.100055,0.255556
9,"April 5th, 2012",0.100041,0.252778
10,"July 5th, 2012",0.099986,0.252778


To get a (theoretical) par bond, we can use a custom index whose conventions match exactly those of the bond we wanted to use: no fixing days, 30/360 day-count convention, and no holidays.  We'll use a curve with the same day-count convention, too.

In [11]:
index = IborIndex('Mock Libor', Period(3, Months), 0, USDCurrency(),
                  NullCalendar(), Unadjusted, False, Thirty360(),
                  forecasting_curve)

bond = FloatingRateBond(settlement_days,
                        face_amount,
                        schedule,
                        index,
                        Thirty360(),
                        Unadjusted,
                        fixingDays = 0)

bond.setPricingEngine(DiscountingBondEngine(discounting_curve))

In [12]:
forecasting_curve.linkTo(flat_rate)
discounting_curve.linkTo(flat_rate)

And now, we finally hit the jackpot:

In [13]:
print(bond.cleanPrice())

100.00000000000001


In [14]:
coupons = [ as_coupon(c) for c in bond.cashflows()[:-1] ]
pd.DataFrame([(c.date(), c.rate(), c.accrualPeriod())
              for c in coupons ],
             columns=('Date', 'Rate', 'Accrual period'),
             index=range(1,len(coupons)+1))

,Date,Rate,Accrual period
1,"April 5th, 2010",0.1,0.25
2,"July 5th, 2010",0.1,0.25
3,"October 5th, 2010",0.1,0.25
4,"January 5th, 2011",0.1,0.25
5,"April 5th, 2011",0.1,0.25
6,"July 5th, 2011",0.1,0.25
7,"October 5th, 2011",0.1,0.25
8,"January 5th, 2012",0.1,0.25
9,"April 5th, 2012",0.1,0.25
10,"July 5th, 2012",0.1,0.25
